Ferramentas de NLP usando o dataset de tweets

Modelo KNN

Pré Processamento

70/30 Treinamento


In [1]:
from google.colab import drive
from collections import Counter
import pandas as pd
from sklearn import tree
import nltk
import re
from sklearn.neighbors import KNeighborsClassifier
from nltk.tokenize.casual import casual_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import train_test_split
from numpy import array
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
import gensim
from gensim.models import Word2Vec

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Dando permissão para o colab poder acessar os arquivos do meu drive

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
df_train = pd.read_csv('/content/drive/My Drive/NLP/DataSet da Primeira Competição/train.csv')
df_test = pd.read_csv('/content/drive/My Drive/NLP/DataSet da Primeira Competição/test.csv')


# df.head()

X_df_test = df_test.Text.values
y_df_test = np.zeros([1640,])

X_df_train = df_train.Text.values
y_df_train = df_train.Classificacao.values


X_df_train = array(X_df_train)
X_df_test = array((X_df_test))
y_df_train = array(y_df_train)
y_df_test = array(y_df_test)

X = np.append(X_df_train,X_df_test)
y = np.append(y_df_train,y_df_test)

# y = pd.get_dummies(y)

In [8]:
df_train.head()

,Created At,Text,Geo Coordinates.latitude,Geo Coordinates.longitude,User Location,Username,User Screen Name,Retweet Count,Classificacao,Observação,Id
0,Mon Jan 09 15:27:43 +0000 2017,Dois são detidos ao tentar jogar celulares e d...,NaN,NaN,NaN,Michele #beta #sdv,michelexmbeta,0,Positivo,NaN,6272
1,Sun Jan 08 02:14:34 +0000 2017,me matan esas minas q cambian 554 veces su fot...,NaN,NaN,Núñez - C.A.B.A.,Gaby Messina,gabymessina36,0,Neutro,NaN,1644
2,Sat Feb 11 09:49:11 +0000 2017,Líderes de motim em presídio de Minas Gerais s...,NaN,NaN,"Hollywood, CA",Wendie Rower,Wendie_Rower,0,Positivo,NaN,7956
3,Thu Jan 05 14:43:03 +0000 2017,#Mídia: Press Release from Business Wire : Di...,NaN,NaN,SP,Marcello Binder,binderbr,0,Neutro,NaN,85
4,Wed Feb 08 22:52:10 +0000 2017,Vacinação contra febre amarela é intensificada...,NaN,NaN,NaN,fodido,eufodeu,0,Positivo,NaN,6006


Para Testar a acuracia no meu conjunto de treino.

In [ ]:
X = df_train.Text
y = df_train.Classificacao

Tentando fazer word2vec

In [ ]:
# iterate through each sentence in the file
# for i in sent_tokenize(X):
#     temp = []
      
#     # tokenize the sentence into words
#     for j in word_tokenize(i):
#         temp.append(j.lower())
  
#     data.append(temp)

data = []

for x in X:
  tokens = word_tokenize(x)
  data.append(tokens)

model1 = gensim.models.Word2Vec(data, min_count = 1, 
                              size = 100, window = 5)

In [ ]:
stop_words_ptbr = nltk.corpus.stopwords.words('portuguese')
stop_words_en = nltk.corpus.stopwords.words('english')
stop_words_spn = nltk.corpus.stopwords.words('spanish')

stop_words = stop_words_ptbr + stop_words_en + stop_words_spn

bags_of_words = []



for x in X:
  x = re.sub(r'http\S+', '', x)
  tokens = casual_tokenize(x, reduce_len=True, strip_handles=True)
  puncs = set((',', '.', '--','-','!','?',':', ';', '``', "''", '(', ')', '[', ']','\u2026','"'))
  tokens = (x.lower() for x in tokens if x not in puncs)
  tokens = Counter(tokens)
  tokens = [x for x in tokens if x not in stop_words]
  bags_of_words.append(Counter(tokens))

df_bows = pd.DataFrame.from_records(bags_of_words)
df_bows = df_bows.fillna(0).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_bows,y, test_size = 0.2, random_state = 109,shuffle=False)

(6559,)

Inverse do OneHotEncode

In [ ]:
# clf = svm.SVC(decision_function_shape="ovr", kernel= "linear")

clf = tree.DecisionTreeClassifier()

clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
predict = clf.predict(X_test)

In [ ]:
df_test.shape

(1640, 10)

In [ ]:
from sklearn.metrics import accuracy_score as acs

print(acs(predict, y_test))

ValueError: ignored

In [ ]:
df_test.Id

0       3568
1       1323
2       7976
3       2408
4       4435
        ... 
1635    3536
1636    6881
1637     627
1638    2165
1639    4268
Name: Id, Length: 1640, dtype: int64

In [ ]:
final = pd.DataFrame(df_test.Id)
final['Category'] = predict

final.to_csv('/content/drive/MyDrive/NLP/DataSet da Primeira Competição/finalTree.csv',index=False)

# df.to_csv('/content/drive/MyDrive/NLP/DataSet da Primeira Competição/final.csv', index=False)

In [ ]:
df

,Category,Id
0,Negativo,3568
1,Neutro,1323
2,Positivo,7976
3,Negativo,2408
4,Negativo,4435
...,...,...
112,Positivo,5279
113,Positivo,6786
114,Negativo,3835
115,Negativo,2859
